<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/%EC%9E%AC%ED%98%84/melon_kakao_eda-%EB%85%B8%EB%9E%98%EC%A0%9C%EA%B1%B01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [5]:
folder = '/content/gdrive/My Drive/melon kakao'
filelist = os.chdir(folder)

In [24]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
song_meta=pd.read_json('song_meta.json')

In [7]:
!git clone https://github.com/slipaway/melonplaylist.git

fatal: destination path 'melonplaylist' already exists and is not an empty directory.


In [17]:
genre_gn_all.head()

GN0100       발라드
GN0101    세부장르전체
GN0102       '80
GN0103       '90
GN0104       '00
dtype: object

In [25]:
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

In [26]:
gnr_code=genre_gn_all[genre_gn_all['gnr_code'].str[-2:]=='00']

In [27]:
#대장르 장르코드 리스트
gnr_code

,gnr_code,gnr_name
0,GN0100,발라드
6,GN0200,댄스
12,GN0300,랩/힙합
18,GN0400,R&B/Soul
22,GN0500,인디음악
32,GN0600,록/메탈
39,GN0700,성인가요
50,GN0800,포크/블루스
56,GN0900,POP
65,GN1000,록/메탈


In [38]:
#곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출

song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

#unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))),
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

#unnested 데이터프레임 생성 : song_gnr_map

song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns=song_gnr_map.columns)
song_gnr_map['id']=song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns={'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace=True)

#unnest 객체 제거
del song_gnr_map_unnest

In [50]:
plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest

In [52]:
plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)

In [53]:
strange_songs

,updt_date,id,issue_date,strange
180,20170915,418694,20180511,True
183,20170915,135950,20180420,True
184,20170915,304687,20180612,True
679,20131017,563802,20161202,True
712,20180102,418694,20180511,True
...,...,...,...,...
5284490,20170414,55899,20170616,True
5285431,20151230,668736,20170601,True
5285478,20151230,91908,20180824,True
5285484,20151230,240770,20180824,True


In [54]:
strange_songs=strange_songs.drop_duplicates(['id'])
len(strange_songs)

2972

In [55]:
strange_songs_list=strange_songs['id'].values.astype(str)
strange_song_gnr_map=song_gnr_map[song_gnr_map['song_id'].isin(strange_songs_list)]

In [56]:
strange_cnt=strange_song_gnr_map.groupby('gnr_code').count()
strange_cnt=pd.merge(strange_cnt, gnr_code, on='gnr_code')

In [57]:
strange_cnt

,gnr_code,song_id,gnr_name
0,GN0100,175,발라드
1,GN0200,46,댄스
2,GN0300,80,랩/힙합
3,GN0400,47,R&B/Soul
4,GN0500,106,인디음악
5,GN0600,64,록/메탈
6,GN0700,14,성인가요
7,GN0800,34,포크/블루스
8,GN0900,760,POP
9,GN1000,491,록/메탈


In [61]:
strange_songs=strange_songs.drop_duplicates(['id'])
print(strange_songs)

        updt_date      id issue_date  strange
180      20170915  418694   20180511     True
183      20170915  135950   20180420     True
184      20170915  304687   20180612     True
679      20131017  563802   20161202     True
934      20160510  306385   20180808     True
...           ...     ...        ...      ...
5264610  20180119  123443   20180209     True
5268374  20100513  663564   20100519     True
5274802  20190824  139433   20191011     True
5278046  20140808  486629   20161202     True
5283397  20060407  398250   20060408     True

[2972 rows x 4 columns]


In [71]:
strange_id=strange_songs.loc[:, 'id']
strange_id
strange_id_list=list(strange_id)

In [83]:
train_id=train.loc[:, 'id']

In [88]:
song_meta_id=song_meta.loc[:, 'id']
print(song_meta_id.head())

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64


In [94]:
strange_songs

,updt_date,id,issue_date,strange
180,20170915,418694,20180511,True
183,20170915,135950,20180420,True
184,20170915,304687,20180612,True
679,20131017,563802,20161202,True
934,20160510,306385,20180808,True
...,...,...,...,...
5264610,20180119,123443,20180209,True
5268374,20100513,663564,20100519,True
5274802,20190824,139433,20191011,True
5278046,20140808,486629,20161202,True


In [95]:
#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map=train[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

In [100]:
#플레이리스트에 수록된 곡의 총 개수(중복 포함)

len(plylst_song_map)

5285871

In [164]:
#플레이리스트에 strange 곡 제거한 후, 수록된 곡의 총 개수(중복 포함)

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))]
plylst_song_map_remove.shape

(5147815, 2)

In [165]:
#train set에 새로운 column : remove_songs 생성
#remove_songs : strange 곡 제거한 후 다시 만든 플레이리스트

plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
train['id']=train['id'].astype(str)
train_remove = pd.merge(train, plylst_song_group)

train_remove.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,remove_songs
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[159327, 553610, 5130, 645103, 294435, 100657,..."
